# STA 141B Lecture 8

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Links to sample projects posted in Project Description
* Assignment 1 grades posted soon

### Topics

* Query Strings
* API Keys
* Undocumented APIs

### Datasets

* [iTunes Search API](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/)
* [The Guardian API](https://open-platform.theguardian.com/)
* [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)


### References

* [__requests__ documentation](http://docs.python-requests.org/en/master/)
* Python for Data Analysis, Ch. 6

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/

## Example Questions

1. Approximately how many remixes are there of PSY's Gangnam Style?
2. Did Clinton or Trump get more newspaper coverage in the days leading up to the 2016 U.S. presidential election?

## Query Strings

Most of the functions we use have parameters, and you can pass arguments for those parameters when you call a function.

Endpoints in REST APIs work the same way, but the syntax is different. You can pass arguments by adding `?PARAMETER=ARGUMENT` to the end of the URL. Parameter and argument pairs are separated by `&`. This syntax is called a _query string_.

For instance, Apple provides a web API for the iTunes store, with [documentation](https://affiliate.itunes.apple.com/resources/documentation/itunes-store-web-service-search-api/). We can use this to answer the example question about Gangnam Style.

The search endpoint is `https://itunes.apple.com/search`, and the documentation lists several parameters. We can use __requests__ to build the query string automatically.

In [117]:
import requests as req
import pandas as pd
import numpy as np
import time

In [126]:
res = requests.get('https://itunes.apple.com/search?',params={'term':'Gangnam Style','country':'US'})
res.url
res.status_code
itunes_data = res.json()
#requests.post('https://itunes.apple.com/search?',data={'key':'value'})

In [130]:
len(itunes_data)
itunes_data.keys()


dict_keys(['resultCount', 'results'])

In [149]:
itunes_data['resultCount']
results = itunes_data['results']
results[0].keys()

dict_keys(['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable'])

In [157]:
df = pd.Series(results[0].keys())
df[1]

dict_keys(['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable'])

Every response has a `.url` attribute that shows the URL used for the request.

In [119]:
res = response.json()["term"]
is_g = results["trackname"].str.contains("Gangnam Style")

NameError: name 'response' is not defined

In [ ]:
time.sleep(0.1)
res_itunes1 = req.get('https://itunes.apple.com/country=US?',params={"api-key":key,"q":"Clinton"})
res_itunes2 = req.get('https://itunes.apple.com/',params={"api-key":key,"country":"US","media":"movie"})
res_itunes2.json()

## Authentication

In [ ]:
requests.post('https://content.guardianapis.com/search?')


### API Keys

Many APIs use a _key_ or _token_ to identify the user.

For instance, The Guardian, a British newspaper, provides a [web API](https://open-platform.theguardian.com/) to access their news articles. You need an API key to use their web APIs. You can get one for free [here](https://bonobo.capi.gutools.co.uk/register/developer).

#### Storing API Keys

Your API key is private and your responsibility. Treat it like a password. Keep it secret! **Don't commit it** in your git repo.

In order to keep your API key separate from your code:
1. Save the API key in a text file.
2. Use Python to load the API key into a variable.

Python's built-in `open()` function opens a file, and the `.readline()` method reads a line from a file. Often you'll see these used with `with`, which automatically closes the file at the end of the block:

In [ ]:
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")

In [ ]:
ls

In [ ]:
# Don't print out your actual API key
print(read_key("keys/guardian_key.txt"))

key = read_key("keys/guardian_key.txt")

In [ ]:
res2 = req.get('https://content.guardianapis.com/search',params={"api-key":key,"q":"Clinton"})

res2.raise_for_status()
res3 = req.get('https://content.guardianapis.com/search',params={"api-key":key,"q":"Clinton","to-date":"2018-12-28","order-date":"last-modified"})
type(res3.json())
data = res3.json()
data.keys()

In [ ]:
data['response'].keys()

In [ ]:
data['response']['status']

In [ ]:
it = data.items()
for key,val in it:
    #print("{} = {}".format(key, val)) 
    pass

Now you can use the `key` variable anywhere you need the actual API key.

#### Querying The Guardian

We've got our key, so let's use The Guardian API to answer our question about media coverage of Clinton and Trump.

Let's start by trying to get all of the articles about one of the candidates.

In [ ]:
def get_article(page = 1):
    res = req.get('https://content.guardianapis.com/search',params={"api-key":key,"q":"Clinton"})
    res.raise_for_status()
    return res.json()

get_article()

What are some ways this analysis could be improved?

### OAuth

[OAuth](https://en.wikipedia.org/wiki/OAuth) is a way to give an application access to data on a website or web API.

You might run into OAuth if you use a web API where the data is private. For instance, Twitter provides a [web API](https://developer.twitter.com/en/docs.html) for managing your personal Twitter account. If you want to access the API from a Python script, first you have to use OAuth to tell Twitter that the script has permission to use your data.

OAuth can operate in several different ways. As always, check the documentation for the web API you want to use in order to find out what you need to do.

The simplest case of OAuth requires scripts to have a key or token from the web API provider. This is very similar to using an API key.

For more complicated cases, the **requests-ouathlib** package (documentation [here](https://requests-oauthlib.readthedocs.io/en/latest/)) may help.

## Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

* [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
* [Yolo County Health Inspections](https://yoloeco.envisionconnect.com/)

You can identify these websites by looking at requests in your browser's developer tools. In Firefox or Chrome, you can open the developer tools with `ctrl-shift-i`.

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

In [ ]:
r1 = req.get('https://ucannualwage.ucop.edu/wage/',params={'year':'2017'})
r1.raise_for_status()
r1

In [ ]:
wage_req = req.get('https://ucannualwage.ucop.edu/wage/aue')
wage_req.json()